I need a very light weight data structure that probably hashes all my reads, and returns just the reads that overlap back to me
Can either be htseq or pysam based, which a preference for pysam, ideally I'd drop this into my c function, but I'll prototype in python first

Given Interval, start, stop, strand

Return all reads overlapping that interval, in O(1) time, really only need two things, read length and number of reads, must take into account splice sites

In [1]:
import HTSeq
from clipper.src.bam_helpers import Robust_BAM_Reader
import pysam
import pandas as pd
from tqdm import tqdm, tqdm_notebook
import numpy as np
import sys
import numpy


In [2]:
bam_file = "/projects/ps-yeolab3/encode/analysis/encode_master/204_01_RBFOX2.merged.r2.bam"
#bam_file = "/projects/ps-yeolab3/encode/analysis/encode_master/240_01_TRA2A.merged.r2.bam"

samfile = pysam.Samfile(bam_file)

In [3]:
chrom_lengths = pd.read_table("/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa.fai", names=['chrom', 'length', 'index', 'len_1', 'len_2'])
chrom_lengths = dict(zip(chrom_lengths.chrom, chrom_lengths["length"]))
chrom_lengths = {"chr11": chrom_lengths['chr11']}

In [4]:
def read_array(reads, start, stop):
    reads = (read for read in reads if (read.iv.start_d > start) & (read.iv.end_d < stop))
    set_of_reads = HTSeq.GenomicArrayOfSets("auto", stranded=True)
    for read in tqdm(reads):
        if read.aligned:
                for cigop in read.cigar:
                    if cigop.type != "M":
                        continue
                    set_of_reads[cigop.ref_iv] += read
    return set_of_reads

In [5]:
start = 65265233
stop = 65273940

In [6]:
reads = Robust_BAM_Reader(bam_file)
reads = list(reads.fetch("chr11", start, stop))


In [7]:
read.qlen

NameError: name 'read' is not defined

In [7]:
#old_version = read_array(reads, start, stop)

In [45]:
reads = Robust_BAM_Reader(bam_file)
reads = list(reads.fetch("chr11", 65265233, 65273940))

reads = (read for read in reads if (read.iv.start_d > start) & (read.iv.end_d < stop))
set_of_reads = HTSeq.GenomicArray(gene_dict, stranded=True, storage="ndarray")
for read in tqdm(reads):
    if read.aligned:
        #read_len = sum(cigar.size for cigar in read.cigar if cigar.type == "M")
        for cigop in read.cigar:
            if cigop.type != "M":
                continue
            cigop.ref_iv.start = cigop.ref_iv.start - start
            cigop.ref_iv.end = cigop.ref_iv.end - start
            set_of_reads[cigop.ref_iv] += read.read.name

35431it [02:30, 235.30it/s]
0it [00:00, ?it/s]
740it [00:00, 7380.30it/s]
962it [00:00, 3931.44it/s]
1174it [00:00, 2406.71it/s]
1360it [00:00, 1699.55it/s]
1521it [00:00, 1247.35it/s]
1657it [00:00, 993.46it/s] 
1772it [00:01, 836.28it/s]
1871it [00:01, 728.29it/s]
1957it [00:01, 657.53it/s]
2034it [00:01, 605.96it/s]
2103it [00:01, 563.98it/s]
2166it [00:01, 527.77it/s]
2224it [00:02, 497.60it/s]
2278it [00:02, 482.61it/s]
2329it [00:02, 462.00it/s]
2378it [00:02, 441.88it/s]
2424it [00:02, 430.27it/s]
2469it [00:02, 411.48it/s]
2512it [00:02, 393.46it/s]
2553it [00:02, 382.21it/s]
2592it [00:03, 371.34it/s]
2630it [00:03, 363.16it/s]
2667it [00:03, 355.09it/s]
2703it [00:03, 350.22it/s]
2739it [00:03, 349.09it/s]
2776it [00:03, 352.76it/s]
2812it [00:03, 347.04it/s]
2847it [00:03, 341.16it/s]
2882it [00:03, 336.31it/s]
2916it [00:03, 330.22it/s]
2950it [00:04, 325.00it/s]
2983it [00:04, 320.34it/s]
3016it [00:04, 315.13it/s]
3048it [00:04, 310.41it/s]
3080it [00:04, 310.20it/s]
3112

KeyboardInterrupt: 

In [30]:
def get_reads_in_interval(interval, array_of_sets):
    return set().union(*[baz for bar, baz in array_of_sets[interval].steps()])

In [31]:
get_reads_in_interval(cigop.ref_iv, set_of_reads)

{('ATTCC:SN1001:449:HGTN3ADXX:2:2114:20422:71492', 42),
 ('TAAAC:SN1001:449:HGTN3ADXX:2:2102:8286:29155', 25),
 ('TTCAG:SN1001:449:HGTN3ADXX:1:1102:11505:40543', 44)}

In [36]:
cigop.ref_iv.start += start
cigop.ref_iv.end += start

In [37]:
get_reads_in_interval(cigop.ref_iv, old_version)

{<SAM_Alignment object: Paired-end read 'TTCAG:SN1001:449:HGTN3ADXX:1:1102:11505:40543' aligned to chr11:[65273511,65273555)/+>,
 <SAM_Alignment object: Paired-end read 'ATTCC:SN1001:449:HGTN3ADXX:2:2114:20422:71492' aligned to chr11:[65273517,65273559)/+>,
 <SAM_Alignment object: Paired-end read 'TAAAC:SN1001:449:HGTN3ADXX:2:2102:8286:29155' aligned to chr11:[65273539,65273564)/+>}

In [14]:
reads = Robust_BAM_Reader(bam_file)
reads = list(reads.fetch("chr11", 65265233, 65273940))

reads = (read for read in reads if (read.iv.start_d > start) & (read.iv.end_d < stop))
set_of_reads = HTSeq.GenomicArrayOfSets("auto", stranded=True)
for read in tqdm(reads):
    if read.aligned:
        read_len = sum(cigar.size for cigar in read.cigar if cigar.type == "M")
        for cigop in read.cigar:
            if cigop.type != "M":
                continue
            cigop.ref_iv.start = cigop.ref_iv.start - start
            cigop.ref_iv.end = cigop.ref_iv.end - start
            set_of_reads[cigop.ref_iv] += read_len

271430it [00:49, 5522.91it/s]


In [9]:
def add_chrom( self, chrom, length = sys.maxint, start_index = 0 ):
  HTSeq.GenomicArray.add_chrom( self, chrom, length, start_index )
  for cv in self.chrom_vectors[ chrom ].values():
    for x in xrange(len(cv.array)):
        cv.array[x] = set([])
    cv.is_vector_of_sets = True
    
def ChromVector_steps( cv ):
   if isinstance( cv.array, numpy.ndarray ):
      start = cv.iv.start
      prev_val = None
      for i in xrange( cv.iv.start, cv.iv.end ):
         val = cv.array[ i - cv.offset ]
         if prev_val is None or val is not prev_val:
            
            if prev_val is not None:
               yield ( HTSeq.GenomicInterval( cv.iv.chrom, start, i, cv.iv.strand ), prev_val )
            prev_val = val
            start = i
      yield ( HTSeq.GenomicInterval( cv.iv.chrom, start, cv.iv.end, cv.iv.strand ), prev_val )
   elif isinstance( cv.array, HTSeq.StepVector.StepVector ):
      for start, stop, value in cv.array[cv.iv.start:cv.iv.end].get_steps():
         yield ( HTSeq.GenomicInterval( cv.iv.chrom, start, stop, cv.iv.strand ), value )
   else:
      raise SystemError, "Unknown array type."

In [10]:
HTSeq._HTSeq_internal.ChromVector_steps = ChromVector_steps
HTSeq.GenomicArrayOfSets.add_chrom = add_chrom


In [38]:
class ChromVector_fast( HTSeq.ChromVector ):
   @classmethod
   def create( cls, iv, typecode, storage, memmap_dir = "" ):
      ncv = cls()
      ncv.iv = iv
      if storage == "ndarray":
         if typecode != 'O':
            ncv.array = numpy.zeros( shape = ( iv.length, ), dtype = typecode )
         else:
            ncv.array = numpy.empty( shape = ( iv.length, ), dtype = typecode )
            ncv.array[:] = None
      elif storage == "memmap":
         ncv.array = numpy.memmap( shape = ( iv.length, ), dtype = typecode,
            filename = os.path.join( memmap_dir, iv.chrom + iv.strand + ".nmm" ), mode='w+' )
      elif storage == "step":
         ncv.array = StepVector.StepVector.create( typecode = typecode )
      else:
         raise ValueError, "Illegal storage mode."
      ncv._storage = storage
      # TODO: Test whether offset works properly
      ncv.offset = iv.start
      ncv.is_vector_of_sets = False
      return ncv

   def __iadd__( self, value ):
      print "going fast"
      if not self.is_vector_of_sets:
         self.array[ self.iv.start - self.offset : self.iv.end - self.offset ].__iadd__( value )
      else:
         def addval( x ):
            x.add( value )
         for iv, value in self.steps():
            addval(value)
      return self

   def __setitem__( self, index, value ):
      print "setting vector"
      if isinstance( value, ChromVector ):
         if self.array is value.array and value.iv.start == index.start and \
               value.iv.end == index.stop and ( index.step is None or index.step == 1 ):
            return
         else:
            raise NotImplementedError, "Required assignment signature not yet implemented."
      if isinstance( index, int ):
         self.array[ index - self.iv.start ] = value
      elif isinstance( index, slice ):
         index_slice = index
         if index_slice.start is not None:
            start = index_slice.start
            if start < self.iv.start:
               raise IndexError, "start too small"
         else:
            start = self.iv.start
         if index_slice.stop is not None:
            stop = index_slice.stop
            if stop > self.iv.end:
               raise IndexError, "stop too large"
         else:
            stop = self.iv.end
         if start > stop:
            raise IndexError, "Start of interval is after its end."
         if start == stop:
            raise IndexError, "Cannot assign to zero-length interval."
         self.array[ start - self.offset : stop - self.iv.start : index.step ] = value
      elif isinstance( index, GenomicInterval ):
         if index.chrom != self.iv.chrom:
            raise KeyError, "Chromosome name mismatch."
         if self.iv.strand is not strand_nostrand and \
               self.iv.strand is not self.index.strand:
            raise KeyError, "Strand mismatch."
         self.array[ index.iv.start - self.iv.start,
            index.iv.end - self.iv.start ] = value
      else:
         raise TypeError, "Illegal index type"


In [39]:
strand_plus = "+"
strand_minus ="-"
strand_nostrand = "."
class GenomicArray_fast( HTSeq.GenomicArray ):
    
   def add_chrom( self, chrom, length = sys.maxint, start_index = 0 ):
      if length == sys.maxint:
         iv = HTSeq.GenomicInterval( chrom, start_index, sys.maxint, "." )
      else:
         iv = HTSeq.GenomicInterval( chrom, start_index, start_index + length, "." )
      if self.stranded:
         self.chrom_vectors[ chrom ] = {}
         iv.strand = "+"
         self.chrom_vectors[ chrom ][ strand_plus ] = \
            ChromVector_fast.create( iv, self.typecode, self.storage, self.memmap_dir )
         iv = iv.copy()
         iv.strand = "-"
         self.chrom_vectors[ chrom ][ strand_minus ] = \
            ChromVector_fast.create( iv, self.typecode, self.storage, self.memmap_dir )
      else:
         self.chrom_vectors[ chrom ] = {
            strand_nostrand:  ChromVector_fast.create( iv, self.typecode, self.storage ) }
            
   def __setitem__( self, index, value ):
      return
      if isinstance( index, HTSeq.GenomicInterval ):
         print "gi detected" 
         if self.stranded and index.strand not in ( strand_plus, strand_minus ):
            raise KeyError, "Non-stranded index used for stranded GenomicArray."
         if self.auto_add_chroms and index.chrom not in self.chrom_vectors:
            self.add_chrom( index.chrom )
         if self.stranded:
            self.chrom_vectors[ index.chrom ][ index.strand ][ index.start : index.end ] = value
         else:
            self.chrom_vectors[ index.chrom ][ strand_nostrand ][ index.start : index.end ] = value
      else:
         raise TypeError, "Illegal index type."

In [40]:
class GenomicArrayOfSets( GenomicArray_fast ):

   """A GenomicArrayOfSets is a specialization of GenomicArray that allows to store                                                                                            
   sets of objects. On construction, the step vectors are initialized with empty sets.                                                                                         
   By using the 'add_value' method, objects can be added to intervals. If an object                                                                                            
   is already present in the set(s) at this interval, an the new object is added to                                                                                            
   the present set, and the set is split if necessary.                                                                                                                         
   """

   def __init__( self, chroms, stranded=True, storage='step', memmap_dir = "" ):
      GenomicArray_fast.__init__( self, chroms, stranded, 'O', storage, memmap_dir )

   def add_chrom( self, chrom, length = sys.maxint, start_index = 0 ):
      GenomicArray_fast.add_chrom( self, chrom, length, start_index )
      for cv in self.chrom_vectors[ chrom ].values():
        for x in xrange(len(cv.array)):
            cv.array[x] = set([])
        cv.is_vector_of_sets = True

In [65]:
for read in samfile:
    if len(read.cigar) != 1:
        break

In [82]:
pysam.AlignedRead = get_full_length_cigar

TypeError: can't set attributes of built-in/extension type 'pysam.libcsamfile.AlignedRead'

In [78]:
read.cigartuples

[(0, 12), (2, 1), (0, 31)]

In [72]:
len(read.query)

43

In [73]:
read.cigarstring

'12M1D31M'

In [75]:
from collections import Counter

In [86]:
def get_full_length_cigar(read):
    for t in read.cigartuples:
        value, times = t
        for x in xrange(times):
            yield value

In [76]:
Counter(read.positions)

Counter({17477: 1,
         17478: 1,
         17479: 1,
         17480: 1,
         17481: 1,
         17482: 1,
         17483: 1,
         17484: 1,
         17485: 1,
         17486: 1,
         17487: 1,
         17488: 1,
         17490: 1,
         17491: 1,
         17492: 1,
         17493: 1,
         17494: 1,
         17495: 1,
         17496: 1,
         17497: 1,
         17498: 1,
         17499: 1,
         17500: 1,
         17501: 1,
         17502: 1,
         17503: 1,
         17504: 1,
         17505: 1,
         17506: 1,
         17507: 1,
         17508: 1,
         17509: 1,
         17510: 1,
         17511: 1,
         17512: 1,
         17513: 1,
         17514: 1,
         17515: 1,
         17516: 1,
         17517: 1,
         17518: 1,
         17519: 1,
         17520: 1})

In [69]:
len(read.positions)

43

In [41]:
set_of_reads = GenomicArrayOfSets(gene_dict, stranded=True, storage="ndarray")

In [42]:
reads = Robust_BAM_Reader(bam_file)
reads = list(reads.fetch("chr11", 65265233, 65273940))

In [43]:
reads = (read for read in reads if (read.iv.start_d > start) & (read.iv.end_d < stop))
for read in reads:
    if read.aligned:
        #read_len = sum(cigar.size for cigar in read.cigar if cigar.type == "M")
        for cigop in read.cigar:
            if cigop.type != "M":
                continue
            cigop.ref_iv.start = cigop.ref_iv.start - start
            cigop.ref_iv.end = cigop.ref_iv.end - start
            set_of_reads[cigop.ref_iv] += read.read.name
    break

IndexError: stop too large

In [51]:
gene_len = stop - start
gene_dict = {"chr11": gene_len}

set_of_reads = GenomicArrayOfSets(gene_dict, stranded=True, storage="ndarray")


In [50]:
list(set_of_reads[cigop.ref_iv])

[{'TCCCG:SN1001:449:HGTN3ADXX:1:2115:2850:77718'},
 {'TCCCG:SN1001:449:HGTN3ADXX:1:2115:2850:77718'}]

In [48]:
from IPython.core.debugger import Tracer

def my_function():
    x = 5
    Tracer()()
    set_of_reads[cigop.ref_iv] += read.read.name

In [49]:
my_function()

/home/gpratt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`


> <ipython-input-48-30221326aa39>(6)my_function()
      2 
      3 def my_function():
      4     x = 5
      5     Tracer()()
----> 6     set_of_reads[cigop.ref_iv] += read.read.name

ipdb> s
--Call--
> <ipython-input-9-f86c948f772b>(8)ChromVector_steps()
      6     cv.is_vector_of_sets = True
      7 
----> 8 def ChromVector_steps( cv ):
      9    if isinstance( cv.array, numpy.ndarray ):
     10       start = cv.iv.start

ipdb> s
> <ipython-input-9-f86c948f772b>(9)ChromVector_steps()
      7 
      8 def ChromVector_steps( cv ):
----> 9    if isinstance( cv.array, numpy.ndarray ):
     10       start = cv.iv.start
     11       prev_val = None

ipdb> n
> <ipython-input-9-f86c948f772b>(10)ChromVector_steps()
      8 def ChromVector_steps( cv ):
      9    if isinstance( cv.array, numpy.ndarray ):
---> 10       start = cv.iv.start
     11       prev_val = None
     12       for i in xrange( cv.iv.start, cv.iv.end ):

ipdb> n
> <ipython-input-9-f86c948f772b>(11)ChromVector_steps()
  

# Lets actually make test hooks now